In [1]:
import os
import tensorflow as tf
from dotenv import load_dotenv
from usd_idr_forecasting.trainers import Retrainer
from usd_idr_forecasting.configs import ProjectConfig
load_dotenv()
PROJECT_WORKING_DIR = os.getenv("PROJECT_WORKING_DIR")

/Users/heykalsayid/Desktop/myown/machine_learning/usd-idr-forecasting/.venv/lib/python3.11/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/heykalsayid/.netrc
wandb: Currently logged in as: heykalsayid (danielteam) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/heykalsayid/.netrc


In [2]:
config_path = os.path.join(PROJECT_WORKING_DIR, 'project_configs.yaml')
project_config = ProjectConfig.from_yaml(config_path)
project_config

ProjectConfig(project_name='idrx-forecast', wandb_team_name='danielteam', general={'windowing_size': 42, 'target_size': 1, 'batch_size': [8, 16, 32], 'shuffle_buffer_size': None}, model={'learning_rate': 0.0005, 'momentum': 0.9, 'optimizer': ['adam'], 'epochs': 50, 'rnn': {'units': 128, 'return_sequences': True}, 'conv1d': {'filter': 32, 'kernel_size': 3, 'strides': 1, 'padding': 'causal', 'activation': 'relu', 'name': 'conv1d'}, 'max_pooling_1d': 42, 'dropout': 0.2, 'feed_forward_layer': {'dense_1': {'dense': 10, 'activation': 'relu', 'name': 'dense_1'}, 'output': {'dense': 1, 'name': 'output'}}}, tuner={'batch_size': 32, 'rnn_units': {'min_value': 50, 'max_value': 200, 'step': 30}, 'conv1d': {'filter': {'min_value': 50, 'max_value': 100, 'step': 25}, 'kernel_size': [3, 42]}, 'max_pool1d': {'name': 'max_pool1d'}, 'flattening_layer': ['GlobalAveragePooling1D', 'GlobalMaxPool1D'], 'batch_norm': {'name': 'batch_norm'}, 'dropout': {'name': 'dropout', 'rate': [0.25, 0.2]}, 'dense_units': [

In [3]:
lstm_retrainer = Retrainer(
	config=project_config, 
	batch_sizes=[16, 32],
	model_rank_ids=[0, 1],
	rnn_type='lstm',
)

In [4]:
# callbacks
reducer_params = dict(
    monitor='val_rmse',
    factor=0.05,
    patience=5,
    verbose=3,
    mode='min',
    min_delta=5e-7,
    cooldown=5,
    min_lr=1e-6,
)

lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(**reducer_params)

In [5]:
lstm_retrainer.start(callbacks=[lr_reducer])

retraining process_id: 2026120_232746_541172


wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb: ERROR The nbformat package was not found. It is required to save notebook history.


5 best LSTM files:
['model-lstm:best-tuned-rank0.keras', 'model-lstm:best-tuned-rank1.keras']
5 best gru files:
['model-gru:best-tuned-rank0.keras', 'model-gru:best-tuned-rank1.keras']


wandb:   37 of 37 files downloaded.  


available preprocessed datasets:
	['train@preprocessed-ds_2025121_172934_626424_batch@8', 'scaler.pkl', 'valid_outliers@preprocessed-ds_2025121_172934_626424_batch@32', 'valid_outliers@preprocessed-ds_2025121_172934_626424_batch@8', 'valid@preprocessed-ds_2025121_172934_626424_batch@8', 'train_outliers@preprocessed-ds_2025121_172934_626424_batch@8', 'valid_outliers@preprocessed-ds_2025121_172934_626424_batch@16', 'valid@preprocessed-ds_2025121_172934_626424_batch@16', 'train@preprocessed-ds_2025121_172934_626424_batch@32', 'train_outliers@preprocessed-ds_2025121_172934_626424_batch@32', 'valid@preprocessed-ds_2025121_172934_626424_batch@32', 'train@preprocessed-ds_2025121_172934_626424_batch@16', 'train_outliers@preprocessed-ds_2025121_172934_626424_batch@16']


...

/Users/heykalsayid/Desktop/myown/machine_learning/usd-idr-forecasting/.venv/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'nadam', because it has 25 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Epoch 1/50
148/148 - 7s - 48ms/step - loss: 0.0345 - mae: 0.1895 - mape: 15323.5947 - mse: 0.0700 - rmse: 0.2635 - val_loss: 0.1049 - val_mae: 0.4465 - val_mape: 51.7693 - val_mse: 0.2102 - val_rmse: 0.4580 - learning_rate: 5.0000e-05
Epoch 2/50
148/148 - 7s - 48ms/step - loss: 0.0321 - mae: 0.1843 - mape: 129713.3750 - mse: 0.0652 - rmse: 0.2543 - val_loss: 0.3638 - val_mae: 0.8242 - val_mape: 94.8603 - val_mse: 0.7369 - val_rmse: 0.8572 - learning_rate: 5.0000e-05
Epoch 3/50
148/148 - 7s - 49ms/step - loss: 0.0273 - mae: 0.1803 - mape: 108267.6250 - mse: 0.0549 - rmse: 0.2336 - val_loss: 2.1402 - val_mae: 2.6429 - val_mape: 306.0902 - val_mse: 7.3041 - val_rmse: 2.6998 - learning_rate: 5.0000e-05
Epoch 4/50
148/148 - 11s - 73ms/step - loss: 0.0234 - mae: 0.1696 - mape: 89581.8750 - mse: 0.0471 - rmse: 0.2162 - val_loss: 6.2481 - val_mae: 6.7545 - val_mape: 783.4720 - val_mse: 47.3964 - val_rmse: 6.8778 - learning_rate: 5.0000e-05
Epoch 5/50

Epoch 5: ReduceLROnPlateau reducing learni

wandb: ERROR The nbformat package was not found. It is required to save notebook history.


epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
epoch/learning_rate,████▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▆▅▃▅▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/mae,█▇▇▆▄▆▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
epoch/mape,▂█▇▆▄▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
epoch/mse,█▆▅▃▅▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/rmse,█▆▅▄▅▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_loss,▁▁▃▆█▅▅▅▅▅▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄
epoch/val_mae,▁▁▃▆█▆▅▅▅▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄
epoch/val_mape,▁▁▃▆█▅▅▅▅▅▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄
+2,...


wandb: Adding directory to artifact (/Users/heykalsayid/Desktop/myown/machine_learning/usd-idr-forecasting/models/retrain/2026120_232746_541172-lstm)... Done. 0.1s
wandb: ERROR The nbformat package was not found. It is required to save notebook history.


In [12]:
lstm_histories = lstm_retrainer.get_model_histories
lstm_histories

{'16_0': <keras.src.callbacks.history.History at 0x123c54bd0>,
 '16_1': <keras.src.callbacks.history.History at 0x1242cbe50>,
 '32_0': <keras.src.callbacks.history.History at 0x124952490>,
 '32_1': <keras.src.callbacks.history.History at 0x124fc2bd0>}